# Adding, Removing, and Changing Boundary Conditions

In [1]:
import openpnm as op
import numpy as np

Start by creating a basic 'demo' network and phase object:

In [2]:
pn = op.network.Demo(shape=[4, 4, 1])
water = op.phase.Water(network=pn)
water.add_model_collection(op.models.collections.physics.standard)
water.regenerate_models()

ModuleNotFoundError: No module named 'thermo'

## Initialize a Transport Algorithm
Now initialize a transport algorithm, in this case diffusion:

In [3]:
fd = op.algorithms.FickianDiffusion(network=pn, phase=water)

NameError: name 'water' is not defined

All transport algorithms have two arrays pre-defined on them for storing **rate** and **value** boundary conditions, ``'pore.bc.rate'`` and ``'pore.bc.value'``. They both initially contain all ``nans`` which is interpreted to mean no boundary conditions are applied in the corresponding pores. 

In [4]:
print(fd['pore.bc.value'])

NameError: name 'fd' is not defined

In [5]:
print(fd['pore.bc.rate'])

NameError: name 'fd' is not defined

**Pro Tip**: Omitting the trailing part of a dictionary key will result in the return of a sub-dictionary containing any arrays who have the requested prefix. This can be used to fetch a dictionary containing all available boundary conditions using ``fd['pore.bc']`` as follows:

In [6]:
print(fd['pore.bc'].keys())

NameError: name 'fd' is not defined

## Setting Value Boundary Conditions

We can apply 'value' or 'Dirchlet' boundary conditions to the pores on the 'left' side of the domain, which is conveniently already labelled by the network generator.  We can assign a value of 1.0 to *all* the pores by passing in a scalar:

In [7]:
fd.set_value_BC(pores=pn.pores('left'), values=1.0, mode='add')

NameError: name 'fd' is not defined

Now when we inspect the 'pore.bc.value' array we can see the 1.0 added to the specified locations:

In [8]:
print(fd['pore.bc.value'])

NameError: name 'fd' is not defined

We can also specify a unique value for each pore by passing in an array:

In [9]:
vals = np.random.rand(pn.num_pores('front'))*10.0
fd.set_value_BC(pores=pn.pores('front'), values=vals, mode='add')

NameError: name 'fd' is not defined

Printing the 'pore.bc.value' array again reveals a problem

In [10]:
print(fd['pore.bc.value'])

NameError: name 'fd' is not defined

Although we added values to 4 locations, only 3 were written, and the pre-existing values of 1.0 all remained. This is because we used ``mode='add'`` for both cases, which does *not* overwrite any pre-existing values. We need to remove the values from the locations first:

In [11]:
fd.set_value_BC(pores=pn.pores('front'), mode='remove')
fd.set_value_BC(pores=pn.pores('front'), values=vals, mode='add')
print(fd['pore.bc.value'])

NameError: name 'fd' is not defined

Now we can see that the 1.0 in pore 0 has been filled with a random number.  There is a convenient short-cut for this using ``mode='overwrite'``:

In [12]:
fd.set_value_BC(pores=pn.pores('front'), values=2.0, mode='overwrite')
print(fd['pore.bc.value'])

NameError: name 'fd' is not defined

**Pro Tip**: The mode argument can be a list of modes, which are each applied in order, so the 'overwrite' behavior can be achieved with ``mode=['remove', 'add']``.  Another useful option may be to clear all existing values the adding new ones, as follows:

In [13]:
fd.set_value_BC(pores=pn.pores('front'), values=2.0, mode=['clear', 'add'])
print(fd['pore.bc.value'])

NameError: name 'fd' is not defined

## Adding Rate Boundary Conditions

Adding rate boundary conditions is done in exactly the same manner as value conditions described above, but via the ``set_rate_BC`` method. One special consideration when using rate boundary conditions is that they *must* be combined with a value boundary condition. Therefore, in this section we'll look at the situation of defining two different types of boundary conditions at the same time. 

When applying rate conditions to pores without any pre-existing value condtions, there is no problem. However, if one wishes to overwrite value BCs with rate BCs (or vice-versa) special treatment is required. To add rate BCs to pores that already have rate BCs one just needs to use ``mode='overwrite'``.  Howver, to add rate BCs to pores with *value* BCs then it is necessary to first remove the values from the target locations since value and rate conditions cannot exist in the same pores.  

Firstly, let's try to add rate BCs and note that nothing is written:

In [14]:
fd.set_rate_BC(pores=pn.pores('front'), rates=2.0, mode='overwrite')
print(fd['pore.bc.rate'])

NameError: name 'fd' is not defined

As can be seen the rate BCs were not written anywhere since value BCs were present in the given pores. If we first ``remove`` the values, then we can get the result we want:

In [15]:
fd.set_value_BC(pores=pn.pores('left'), mode='remove')
fd.set_rate_BC(pores=pn.pores('left'), rates=2.0, mode='overwrite')
print(fd['pore.bc.rate'])

NameError: name 'fd' is not defined

## Removing and Resetting Boundary Conditions

The above example illustrated how it was necessary to remove the value BCs from the target pores prior to adding the rate BCs. The required knowing which types were defined and each needed to be removed explicitly.  An alternative and more general approach is possible.

**Pro Tip**: A list of BC types can be passed ``set_BC``, which will each be handled in order. If the mode happens to be ``clear``, then all the given bc types will be cleared.  Combining this with the dictionary trick used above makes it possible to remove all the bcs without knowing which types are present.  This is illustrated as follows:

In [16]:
fd.set_BC(bctype=fd['pore.bc'].keys(), mode='clear')

NameError: name 'fd' is not defined

Now all the bc arrays will be filled with ``nan`` meaning that no conditions are set:

In [17]:
print(fd['pore.bc.rate'])

NameError: name 'fd' is not defined

In [18]:
print(fd['pore.bc.value'])

NameError: name 'fd' is not defined

**Pro Tip**: It is possible to edit the bc arrays directly if you know what you're doing.  For instance, setting value BCs in the left and right pores can be done as follows:

In [19]:
fd['pore.bc.value'][pn.pores('left')] = 1.0
fd['pore.bc.value'][pn.pores('right')] = 0.0

NameError: name 'fd' is not defined

Running the algorithm will proceed as normal:

In [20]:
soln = fd.run()
print(soln)

NameError: name 'fd' is not defined

And of course, these bcs can be "reset" directly too:

In [21]:
fd['pore.bc.value'] = np.nan

NameError: name 'fd' is not defined

Where the above will expand the scalar value (i.e. ``nan``) and apply it to all pores.  